In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch2
(
  mch2_id STRING,
  mch2_name STRING,
  mch2_desc STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""CREATE OR REPLACE TEMP VIEW temp_d_mch2 AS
SELECT 
a._bic_zc_mchlv2 AS mch2_id,  
a.txtsh AS mch2_name,
a.txtmd AS mch2_desc
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_mchlv2 ORDER BY proc_date DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_mchlv2
        WHERE end_date = '2400-01-01'
    ) a WHERE rn = 1 
;""")

In [0]:
spark.sql(
    f"""MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch2 AS main_tbl
USING temp_d_mch2 AS temp_tbl
ON main_tbl.mch2_id = temp_tbl.mch2_id 

WHEN MATCHED THEN 
    UPDATE SET 
        main_tbl.mch2_name = temp_tbl.mch2_name,
        main_tbl.mch2_desc = temp_tbl.mch2_desc

WHEN NOT MATCHED THEN 
    INSERT (mch2_id, mch2_name, mch2_desc)
    VALUES (temp_tbl.mch2_id, temp_tbl.mch2_name, temp_tbl.mch2_desc)""")

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_d_mch2")